---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 3

In this assignment you will explore measures of centrality on two networks, a friendship network in Part 1, and a blog network in Part 2.

## Part 1

Answer questions 1-4 using the network `G1`, a network of friendships at a university department. Each node corresponds to a person, and an edge indicates friendship. 

*The network has been loaded as networkx graph object `G1`.*

In [5]:
######################
#      Influence Measures and Network Centralizationの課題      #
# Part1: 小規模(ローカル)ネットワーク
# Q1.　Centralityのアルゴリズム３種の値を求める
# Q2. 枚数は制限なしで１度だけ他のノードに渡せる（渡しても有効）クーポンをどの人に渡すべきか
# Q3. クーポンを経由する人の数を少なくしながら多くの人に配りたい時どの人に渡すべきか
# Q4. 競合他社がサービスの邪魔をしようとネットワークから中核人物（最も情報をよく経由する人）を引き抜こうとしている。誰が狙われているか？
#
# Part2: 大規模(Web)ネットワーク
# Q5. あるウェブサイトのPageRankを求める
# Q6. PageRankアルゴリズムを使って最もランクの高い５つのウェブサイトを見つけ出す
# Q7. あるウェブサイトのHitsを求める
# Q8. Hitsアルゴリズムを使って最もHubスコアの高い５つのウェブサイトを見つけ出す
# Q9. Hitsアルゴリズムを使って最もAuthスコアの高い５つのウェブサイトを見つけ出す
######################


#####!!REVIEW!!##
# ◆ Degree and Closeness Centrality ◆
# degree centrality の想定したこと :  他のノードとの繋がりが多いほど重要なノード
# closeness centrality の想定したこと : 他のノードとの平均距離が短いほど重要なノード
# betweenness centrality の想定したこと : 他のノードがshortest pathでよく通過するノードほど重要なノード
# 
# degree centrality( of node)の式 : degreeの数 ÷ (ノードの総数 - 1)
#                                    G = nx.convert_labels_to_integers(G, first_label=1)
#                                    degCent = nx.degree_centrality(G)
#                                    print(degCent[34])  => 例: 0.515 (17 / 33)(※全部で34ノード)
# in-degree centrality : Directed Graphのin-degree　　式 = in degreeの数 ÷ (ノードの総数 - 1)
#                                    indegCent = nx.degree_centrality(G)
# out-degree centrality : Directed Graphのout-degree　　式 = out degreeの数 ÷ (ノードの総数 - 1)
#                                    outdegCent = nx.out_degree_centrality(G)
#
# closeness centrality( of node)の式 : (ノードの総数 - 1) ÷ (距離の和)　　※ 距離の和 ==> sum(nx.shortest_path_length(G, 32).values())
#                                    closeCent = nx.closeness_centrality(G)
#                                    print(degCent[32])  => 例: 0.541 (len(G.nodes()) -1) ÷ sum(nx.shortest_path_length(G, 32).values()) で求まる。
#                                    Disconnected Nodes(他に到達できないノード)はどうするか。
#                                    →案1. ノードが到達できるノードのみをネットワークとして考える。
#                                        closeCent = nx.closeness_centrality(G, normalized = False)
#                                    →案2. ノードが到達できるノードのみをネットワークとして考え、(その数 / (N-1))を掛けて全体で値が狂わないようにNormalizeする。
#                                        closeCent = nx.closeness_centrality(G, normalized = False)
#
# ◆ Betweenness Centrality ◆
# normalized betweenness centrality( of node)の式 : sum(各ノード間のshortest_pathでノードを通過する数/ 各ノード間のshortest_pathの数)
#                                                                                         (2点間のshortest_pathは何通りもあるのでこのような式になる, nodeでなくpathで計算する場合もある)
#                                            Disconnected Nodes(他に到達できないノード)はどうするか。
#                                            →案1. そのノード間の式を(sum関数に)含めない。
#
# betweenness centrality と normalizationの関係 : ノード数が増えてくると値は必然的に多くなるので公平の為にノードのペアの数で割る(大きなグラフほど大きな値になるのを防ぐ為)
#                                             Unirected Graph=> 1/2 *  (ノードの総数 - 1) * (ノードの総数 - 2)
#                                             Directed Graph=> (ノードの総数 - 1) * (ノードの総数 - 2)
#                                    btwnCent = nx.betweenness_centrality(G,
#                                                               normalized=True, endpoints=False) # centralityのノードを含める時はendpoints=True
#                                    import operator
#                                    sorted(btwnCent.items(), key=operator.itemgetter(1), reverse=True)[:5] # 上位5つのノードが得られる
#                                    
#  <上記まとめ>                                 
# Normalization : Divide by number of pairs of nodes
# Approximation : Computing betweenness centrality can be computationally expensive. (2200ノードで500万ペアになるから)
#                              We can approximate computation by taking a subset of nodes.
# 　　　　　　　　　　　　　　　　　btwnCent_approx = nx.betweenness_centrality(G,
#                                                        normalized=True, endpoints=False, k=10) 
# Subsets : We can define subsets of source and target nodes to compute betweenness centrality.(これもコンピュートコストの高騰を抑える為の方法)
# 　　　　　　　　　　　　　　　　　btwnCent_subset = nx.betweenness_centrality_subset(G,
#                                                        [1,2,3,4],[5,6,7,8], normalized=True) #  [1,2,3,4]=>source subset nodes, [5,6,7,8]=>target subset nodes
# Edge betweenness centrality : We can apply the same framework to find important edges instead of nodes.　どのパスがよく通過するパスか
# 　　　　　　　　　　　　　　　　　btwnCent_edge = nx.edge_betweenness_centrality(G, normalized=True) 
#                   import operator
#                   sorted(btwnCent_edge.items(), key=operator.itemgetter(1), reverse=True)[:5] # 上位5つのedgeが得られる
#
# ◆ Page Rank ◆
# Page Rankの想定したこと : in-linkの多いノードほど重要なノード、そしてin-linkの多いノードからout-linkされているノードも重要なノード
#　　　　　　　　　　　n: ノードの総数, k: ステップ数として     (循環して求める為)
#　　　　　　　　　　　Basic Page Rank:
#　　　　　　　　　　　　　　　　　    ⅰ. 最初に全ノードのPage Rankを 1/n と定義する
#　　　　　　　　　　　　　　　　   　ⅱ. Page Rankを求める: sum((1/ リンク元のout linkの数 )* リンク元のPageRank) ... ※1
#　　　　　　　　　　　　　　　　　  ⅲ. k回繰り返す
# ※1: 例えばk=1の時、ノードAにout linkしているノードが他のノード2つにもout linkしていれば、ノードAのPage Rankは1/3 * 1/n が加算される。
#            Scaled Page Rank:
#                      ⅰ. k=1の段階で全てのノードを周回して計算しない。ランダムに一定数探索し終えるとkをカウントアップする方法。
#                     ⅱ. α (Damping Parameter) : Scaled Page Rankにおいてクローラなどが袋小路に入らないように、αの確率で次へ進むが、1-αの確率でランダムなノードに飛ばすこと。
#                                                                    一般的に0.8~0.9の値を使用する。
#
# ◆ Hits (Page Rankとそっくりだけど別のアルゴリズム) ◆
# Authority = 他からどれだけリンクを受けて権威を当てられているか。 
# Hub = どれだけリンクを持ちhubの役割を果たしているか
# HITS : 各ノードにAuthority スコア, Hub スコアを 割振るアルゴリズム。
#　　　　　　　　　　　　　　　　　    ⅰ. 最初に全ノードのauthority score, hub score を 1 と定義する
#　　　　　　　　　　　　　　　　   　ⅱ. Authority Update Rule: そのノードに向けてlinkしているHub scoreの合計をそのノードのAuthority scoreに更新する
#　　　　　　　　　　　　　　　　   　ⅲ. Hub Update Rule: そのノードがlinkしているノードのAuthority scoreの合計をそのノードのHub scoreに更新する
#　　　　　　　　　　　　　　　　　   ⅳ. そのノード(j)に対しNormalizeする 式 : 「auth(j) = auth(j) / 全ノードのauth合計」 (... authは絶え間なく増えていくので。)
#　　　　　　　　　　　　　　　　　  ⅴ. k回繰り返す
#
# 
#
#######
import networkx as nx

G1 = nx.read_gml('friendships.gml')
print(G1.nodes()[:5])

[1, 2, 3, 4, 5]


### Question 1

Find the degree centrality, closeness centrality, and normalized betweeness centrality (excluding endpoints) of node 100.

*This function should return a tuple of floats `(degree_centrality, closeness_centrality, betweenness_centrality)`.*

In [7]:
def answer_one():
        
    # Your Code Here
    degCent = nx.degree_centrality(G1)[100]
    closeCent = nx.closeness_centrality(G1)[100]
    btwnCent = nx.betweenness_centrality(G1)[100] # 遅い
    
    return degCent, closeCent, btwnCent# Your Answer Here
answer_one()

(0.0026501766784452294, 0.2654784240150094, 7.142902633244772e-05)

<br>
#### For Questions 2, 3, and 4, assume that you do not know anything about the structure of the network, except for the all the centrality values of the nodes. That is, use one of the covered centrality measures to rank the nodes and find the most appropriate candidate.
<br>

### Question 2

Suppose you are employed by an online shopping website and are tasked with selecting one user in network G1 to send an online shopping voucher to. We expect that the user who receives the voucher will send it to their friends in the network.  You want the voucher to reach as many nodes as possible. The voucher can be forwarded to multiple users at the same time, but the travel distance of the voucher is limited to one step, which means if the voucher travels more than one step in this network, it is no longer valid. Apply your knowledge in network centrality to select the best candidate for the voucher. 

*This function should return an integer, the name of the node.*

In [4]:
def answer_two():
        
    # Your Code Here
    # degree centrality の想定したこと :  他のノードとの繋がりが多いほど重要なノード
    degCent = nx.degree_centrality(G1)
    import operator
    list_rank = sorted(degCent.items(), key=operator.itemgetter(1), reverse=True)
    print(list_rank[:5])
    
    
    return list_rank[0][0]# Your Answer Here
answer_two()

[(105, 0.0636042402826855), (23, 0.045936395759717315), (333, 0.045936395759717315), (16, 0.045053003533568906), (42, 0.045053003533568906)]


105

### Question 3

Now the limit of the voucher’s travel distance has been removed. Because the network is connected, regardless of who you pick, every node in the network will eventually receive the voucher. However, we now want to ensure that the voucher reaches the nodes in the lowest average number of hops.

How would you change your selection strategy? Write a function to tell us who is the best candidate in the network under this condition.

*This function should return an integer, the name of the node.*

In [6]:
def answer_three():
        
    # Your Code Here
    # closeness centrality の想定したこと : 他のノードとの平均距離が短いほど重要なノード
    closeCent = nx.closeness_centrality(G1)
    import operator
    list_rank = sorted(closeCent.items(), key=operator.itemgetter(1), reverse=True)
    
    return list_rank[0][0]# Your Answer Here
answer_three()

23

### Question 4

Assume the restriction on the voucher’s travel distance is still removed, but now a competitor has developed a strategy to remove a person from the network in order to disrupt the distribution of your company’s voucher. Your competitor is specifically targeting people who are often bridges of information flow between other pairs of people. Identify the single riskiest person to be removed under your competitor’s strategy?

*This function should return an integer, the name of the node.*

In [7]:
def answer_four():
        
    # Your Code Here
    # betweenness centrality の想定したこと : 他のノードがshortest pathでよく通過するノードほど重要なノード
    btwnCent = nx.betweenness_centrality(G1)
    import operator
    list_rank = sorted(btwnCent.items(), key=operator.itemgetter(1), reverse=True)
    

    return list_rank[0][0]# Your Answer Here
answer_four()

333

## Part 2

`G2` is a directed network of political blogs, where nodes correspond to a blog and edges correspond to links between blogs. Use your knowledge of PageRank and HITS to answer Questions 5-9.

In [8]:
G2 = nx.read_gml('blogs.gml')

### Question 5

Apply the Scaled Page Rank Algorithm to this network. Find the Page Rank of node 'realclearpolitics.com' with damping value 0.85.

*This function should return a float.*

In [12]:
def answer_five():
        
    # Your Code Here
    pr = nx.pagerank(G2, alpha=0.85)
    
    return pr['realclearpolitics.com']# Your Answer Here
answer_five()

0.004636694781649093

### Question 6

Apply the Scaled Page Rank Algorithm to this network with damping value 0.85. Find the 5 nodes with highest Page Rank. 

*This function should return a list of the top 5 blogs in desending order of Page Rank.*

In [20]:
def answer_six():
        
    # Your Code Here
    pr = nx.pagerank(G2, alpha=0.85)
    pagerank_list = [ (k,v) for k,v in pr.items() ]
    sorted_pr_list = sorted(pagerank_list, key=lambda x: x[1], reverse=True)
    
    top_five_sites = [k for k,v in sorted_pr_list[:5]]
    
    return top_five_sites# Your Answer Here
answer_six()

['dailykos.com',
 'atrios.blogspot.com',
 'instapundit.com',
 'blogsforbush.com',
 'talkingpointsmemo.com']

### Question 7

Apply the HITS Algorithm to the network to find the hub and authority scores of node 'realclearpolitics.com'. 

*Your result should return a tuple of floats `(hub_score, authority_score)`.*

In [22]:
def answer_seven():
        
    # Your Code Here
    hubs, auths = nx.hits(G2)

    
    return hubs['realclearpolitics.com'], auths['realclearpolitics.com']# Your Answer Here
answer_seven()

(0.000324355614091667, 0.003918957645699856)

### Question 8 

Apply the HITS Algorithm to this network to find the 5 nodes with highest hub scores.

*This function should return a list of the top 5 blogs in desending order of hub scores.*

In [24]:
def answer_eight():
        
    # Your Code Here
    hubs, auths = nx.hits(G2)
    hub_rank_list = [ (k,v) for k,v in hubs.items() ]
    sorted_hub_rank_list = sorted(hub_rank_list, key=lambda x: x[1], reverse=True)
    
    top_five_sites = [k for k,v in sorted_hub_rank_list[:5]]
    
    return top_five_sites# Your Answer Here
answer_eight()

['politicalstrategy.org',
 'madkane.com/notable.html',
 'liberaloasis.com',
 'stagefour.typepad.com/commonprejudice',
 'bodyandsoul.typepad.com']

### Question 9 

Apply the HITS Algorithm to this network to find the 5 nodes with highest authority scores.

*This function should return a list of the top 5 blogs in desending order of authority scores.*

In [25]:
def answer_nine():
        
    # Your Code Here
    hubs, auths = nx.hits(G2)
    auth_rank_list = [ (k,v) for k,v in auths.items() ]
    sorted_auth_rank_list = sorted(auth_rank_list, key=lambda x: x[1], reverse=True)
    
    top_five_sites = [k for k,v in sorted_auth_rank_list[:5]]
    
    return top_five_sites# Your Answer Here
answer_nine()

['dailykos.com',
 'talkingpointsmemo.com',
 'atrios.blogspot.com',
 'washingtonmonthly.com',
 'talkleft.com']